## Dataset Exploration

In [3]:
import os 
#make sure your working directory is the repository root. 
if not os.getcwd().endswith("idp-radio-1"):
    os.chdir("../../")
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
os.getcwd()

'/srv/idp-radio-1'

In [4]:
load_dotenv(find_dotenv())
chestxray14_folder = Path(os.environ.get("CHESTXRAY14_DATASET_DIRECTORY"))
chestxray14_folder

PosixPath('data/chestxray14/scale_256')

In [7]:

labels = pd.read_csv(chestxray14_folder / "meta/data/labels.csv")
labels.columns

Index(['Image Index', 'Follow-up #', 'Patient ID', 'Patient Age',
       'Patient Gender', 'View Position', 'OriginalImage_Width',
       'OriginalImage_Height', 'OriginalImagePixelSpacing_x',
       'OriginalImagePixelSpacing_y', 'Part of Train and Val Set',
       'Part of Test Set', 'bounding box', 'Edema', 'Atelectasis', 'Pneumonia',
       'Pleural_Thickening', 'Cardiomegaly', 'Infiltration', 'Consolidation',
       'Fibrosis', 'No Finding', 'Effusion', 'Nodule', 'Mass', 'Hernia',
       'Emphysema', 'Pneumothorax'],
      dtype='object')